### How to install C++ kernel in Jupyter? 
1. Install miniconda
2. Create a new environment: ```conda create -n cpp_env```
<!-- (3. Initialise the environment: ```conda init```) -->
4. Activate the environment: ```conda activate cpp_env```
5. Install xeus-cling: ```conda install jupyter xeus-cling -c conda-forge```
<!--6. Launch Jupyter: ```jupyter notebook --no-browser```
7. Change kernel, select another kernel, existing jupyter server and copy-paste (if needed) the url provided in the terminal -->
8. Change kernel and select a C++ kernel

# Option pricing

This notebook presents different methods to calculate option prices. \
For the moment, it only supports European and American options. 

In [1]:
#include <cmath>
#include <iostream>
#include <string>
#include <algorithm>
#include <stdexcept>
#include <random>
using namespace std;

In [2]:
double St=100;
double K=100;
double T=1.;
double t=0;
double r=.05;
double q=0.5;
double sigma=.2;

### Black-Scholes
$$C(S_t, t) = N(d_+)S_t e^{-q(T - t)} - N(d_-)Ke^{-r(T - t)}$$
$$d_+ = \frac{1}{\sigma\sqrt{T - t}}\left[\ln\left(\frac{S_t}{K}\right) + \left(r-q + \frac{\sigma^2}{2}\right)(T - t)\right] $$
$$     d_- = d_+ - \sigma\sqrt{T - t} $$

In [ ]:
/**
 * @brief Approximates the standard normal cumulative distribution function (CDF) from -∞ to a given point using the Riemann sum method.
 *
 * This function calculates the integral of the standard normal probability density function (PDF) from -50 to `b` using a Riemann sum approximation.
 * The integral is approximated by summing the area of rectangles under the curve of the standard normal PDF.
 *
 * @param b The upper limit of integration for the CDF approximation.
 * @return double The approximated value of the standard normal CDF at `b`.
 *
 * @note
 * - The lower limit of integration is fixed at -50, which is effectively -∞ for the standard normal distribution.
 * - The number of subintervals for the Riemann sum is fixed at 100,000 for accuracy.
 * - The function uses the standard normal PDF: \( \frac{1}{\sqrt{2\pi}} e^{-x^2/2} \).
 */
double normal_cdf_riemann(double b) {
    // Riemann sum approximation of the normal CDF from -infinity to b
    double a = -50; // Limit of integration
    int n = 100000; // Number of subintervals
    double dx = (b - a) / n;
    double s = 0.0;
    double x = a;
    for (int i = 0; i < n; i++) {
        s += exp(-pow(x, 2) / 2) * dx;
        x += dx;
    }
    return s / sqrt(2 * M_PI);
}


In [ ]:
/**
 * @brief Computes the price of a European call or put option using the Black-Scholes model.
 *
 * @param call_put A string indicating the type of option: "call" or "put".
 * @param sigma The volatility of the underlying asset (annualized standard deviation of returns).
 * @param K The strike price of the option.
 * @param T The time to maturity of the option (in years).
 * @param t The current time (in years).
 * @param St The current price of the underlying asset.
 * @param r The risk-free interest rate (annualized, continuously compounded).
 * @param q The dividend yield of the underlying asset (annualized, continuously compounded).
 *
 * @return double The price of the European call or put option.
 *
 * @throws runtime_error If the `call_put` parameter is not "call" or "put".
 *
 * @details
 * The Black-Scholes model is used to calculate the theoretical price of European-style options.
 * The formula for a call option is:
 * \[
 * C = S_t e^{-q(T-t)} N(d_+) - K e^{-r(T-t)} N(d_-)
 * \]
 * The formula for a put option is:
 * \[
 * P = K e^{-r(T-t)} N(-d_-) - S_t e^{-q(T-t)} N(-d_+)
 * \]
 * where:
 * \[
 * d_+ = \frac{\ln(S_t/K) + (r - q + \sigma^2/2)(T-t)}{\sigma \sqrt{T-t}}
 * \]
 * \[
 * d_- = d_+ - \sigma \sqrt{T-t}
 * \]
 * and \( N(x) \) is the cumulative distribution function of the standard normal distribution.
 */
double Black_Scholes(string call_put, double sigma, double K, double T, double t, double St, double r, double q) {
    // Black-Scholes formula for European options
    double d_plus = (log(St/K)+(r-q+pow(sigma,2)/2)*(T-t))/(sigma*sqrt(T-t));
    double d_minus = d_plus - sigma*sqrt(T-t);
    if(call_put=="call"){
        return normal_cdf_riemann(d_plus)*St*exp(-q*(T-t))- normal_cdf_riemann(d_minus)*K*exp(-r*(T-t));
    }
    else if (call_put=="put"){
        return -normal_cdf_riemann(-d_plus)*St*exp(-q*(T-t))+ normal_cdf_riemann(-d_minus)*K*exp(-r*(T-t));
    }
    else{
        throw runtime_error("Please choose between call and put");
    }
}


In [5]:
cout << "Black-Scholes call price: "+to_string(Black_Scholes("call", sigma, K, T, t, St, r, q)) << endl;
cout << "Black-Scholes put price: "+to_string(Black_Scholes("put", sigma, K, T, t, St, r, q)) << endl;

Black-Scholes call price: 0.064067
Black-Scholes put price: 34.533943


### Monte-Carlo simulation

In [ ]:
/**
 * @brief Estimates the price of a European call or put option using Monte Carlo simulation.
 *
 * @param call_put A string indicating the type of option: "call" or "put".
 * @param sigma The volatility of the underlying asset (annualized standard deviation of returns).
 * @param K The strike price of the option.
 * @param T The time to maturity of the option (in years).
 * @param t The current time (in years).
 * @param St The current price of the underlying asset.
 * @param r The risk-free interest rate (annualized, continuously compounded).
 * @param q The dividend yield of the underlying asset (annualized, continuously compounded).
 *
 * @return double The estimated price of the European call or put option, discounted to present value.
 *
 * @throws runtime_error If the `call_put` parameter is not "call" or "put".
 *
 * @details
 * This function uses a Monte Carlo simulation to estimate the price of a European option.
 * It simulates `N` paths of the underlying asset price using geometric Brownian motion:
 * \[
 * S_{t+dt} = S_t \exp\left(\left(r - q - \frac{\sigma^2}{2}\right)dt + \sigma \sqrt{dt} \, Z\right)
 * \]
 * where \( Z \) is a standard normal random variable.
 * The option price is estimated as the average discounted payoff across all simulated paths.
 * The number of time steps is fixed at 10, and the number of paths is fixed at 50,000.
 */
double Monte_Carlo(string call_put, double sigma, double K, double T, double t, double St, double r, double q) {
    // Monte Carlo simulation for European options
    default_random_engine generator; // Create a generator
    normal_distribution<double> distribution(0,1); // Specify the distribution
    int n=10; // Number of time points
    double dt=(T-t)/(n-1);
    int N=50000; // Number of paths
    double S[N][n];
    for (int i=0; i<N; i++){
      S[i][0]=St; // Initialize all the paths at St
    }
    for (int i = 0; i < N; i++) {
      for (int j=1; j<n; j++){
        S[i][j] = S[i][j-1]*exp((r-q-pow(sigma,2)/2)*dt+sigma*sqrt(dt)*distribution(generator)); // Simulate the paths
      }
    }
    double payoff_sum = 0.0;
    if (call_put=="call"){
        for (int i=0; i<N; i++){
            payoff_sum += max(S[i][n-1]-K,0.);  // Calculate the payoff for call
        }
    }
    else if(call_put=="put"){
        for (int i=0; i<N; i++){
            payoff_sum += max(K-S[i][n-1],0.);  // Calculate the payoff for put
        }
    }
    else{
        throw runtime_error("Please choose between call and put");
    }
    return payoff_sum/N*exp(-r*(T-t)); // Return the average discounted payoff
}


In [7]:
cout << "Monte-Carlo European call price: "+to_string(Monte_Carlo("call", sigma, K, T, t, St, r, q)) << endl;
cout << "Monte-Carlo European put price: "+to_string(Monte_Carlo("put", sigma, K, T, t, St, r, q)) << endl;

Monte-Carlo European call price: 0.063259
Monte-Carlo European put price: 34.588330


### Tree

In [ ]:
/**
 * @brief Prices American or European call/put options using a binomial tree model.
 *
 * @param call_put A string indicating the type of option: "call" or "put".
 * @param am_eu A string indicating the exercise style: "American" or "European".
 * @param sigma The volatility of the underlying asset (annualized standard deviation of returns).
 * @param K The strike price of the option.
 * @param T The time to maturity of the option (in years).
 * @param t The current time (in years).
 * @param St The current price of the underlying asset.
 * @param r The risk-free interest rate (annualized, continuously compounded).
 * @param q The dividend yield of the underlying asset (annualized, continuously compounded).
 *
 * @return double The price of the American or European call/put option.
 *
 * @throws runtime_error If `call_put` is not "call" or "put", or if `am_eu` is not "American" or "European".
 *
 * @details
 * This function uses a binomial tree model to price options. The tree is constructed with `n` time steps.
 * The up and down factors are calculated as:
 * \[
 * U = e^{\sigma \sqrt{dt}}, \quad D = \frac{1}{U}
 * \]
 * where \( dt = \frac{T-t}{n} \).
 * The risk-neutral probability \( p \) is given by:
 * \[
 * p = \frac{e^{(r-q)dt} - D}{U - D}
 * \]
 * For European options, the price at each node is the discounted expected value of the option at the next time step.
 * For American options, the price at each node is the maximum of the discounted expected value and the immediate exercise value.
 * The tree recombines, resulting in \( i+1 \) nodes at each time step \( i \).
 */
double Tree(string call_put, string am_eu, double sigma, double K, double T, double t, double St, double r, double q) {
    // Binomial tree for American and European options
    int n=150; // Number of time periods
    double dt=(T-t)/n;
    double S[n][n+1];
    double U=exp(sigma*sqrt(dt));
    double D=1/U;
    S[0][0]=St; // Initialize the tree

    // Build the stock price tree
    for (int i = 0; i<n-1; i++) {
      for (int j=0; j<i+1; j++){
        S[i+1][j] = S[i][j]*U; // Up movement
      }
      S[i+1][i+1]=S[i][i]*D; // Down movement for the last node
    }

    // Initialize option prices at maturity
    double prices[n][n+1];
    if(call_put=="call"){
      for (int j=0; j<n+1; j++) {
        prices[n-1][j]=max(S[n-1][j]-K,0.); // Payoff at maturity for call
      }
    }
    else if(call_put=="put"){
      for (int j=0; j<n+1; j++) {
        prices[n-1][j]=max(K-S[n-1][j],0.); // Payoff at maturity for put
      }
    }
    else{
        throw runtime_error("Please choose between call and put");
    }

    // Risk-neutral probability
    double p=(exp((r-q)*dt)-D)/(U-D);

    // Backward induction for European options
    if(am_eu=="European"){
      for (int i = n-2; i>=0; i--) {
        for (int j=0; j<i+1; j++) {
          prices[i][j] = (p*prices[i+1][j]+(1-p)*prices[i+1][j+1])*exp(-r*dt);
        }
      }
    }
    // Backward induction for American options
    else if(am_eu=="American"){
      if(call_put=="call"){
        for (int i = n-2; i>=0; i--) {
          for (int j=0; j<i+1; j++) {
            prices[i][j] = max((p*prices[i+1][j]+(1-p)*prices[i+1][j+1])*exp(-r*dt), S[i][j]-K); // Early exercise adjustment for call
          }
        }
      }
      else if(call_put=="put"){
        for (int i = n-2; i>=0; i--) {
          for (int j=0; j<i+1; j++) {
            prices[i][j] = max((p*prices[i+1][j]+(1-p)*prices[i+1][j+1])*exp(-r*dt), K-S[i][j]); // Early exercise adjustment for put
          }
        }
      }
    }
    else {
        throw runtime_error("Please choose American or European.");
    }
    return prices[0][0];
}


In [9]:
cout << "Binomial tree European call price: "+to_string(Tree("call", "European", sigma, K, T, t, St, r, q)) << endl;
cout << "Binomial tree European put price: "+to_string(Tree("put", "European", sigma, K, T, t, St, r, q)) << endl;

cout << "Binomial tree American call price: "+to_string(Tree("call", "American", sigma, K, T, t, St, r, q)) << endl;
cout << "Binomial tree American put price: "+to_string(Tree("put", "American", sigma, K, T, t, St, r, q)) << endl;

Binomial tree European call price: 0.062082
Binomial tree European put price: 34.361158
Binomial tree American call price: 1.556477
Binomial tree American put price: 34.361158


### PDE and finite difference methods

$$\frac{\partial V}{\partial t} + \frac{1}{2}\sigma^2 S^2 \frac{\partial^2 V}{\partial S^2} +  (r-q)S\frac{\partial V}{\partial S} -  rV = 0$$
$$\Theta + \frac{1}{2}\sigma^2 S^2 \Gamma +  (r-q)S \Delta -  rV = 0$$
We can discretize the Greeks (I choose to use the explicit method, ie backward approximation for $\Theta$, central approximation for $\Delta$ and standard approximation for $\Gamma$):
$$\Theta =\frac{\partial V}{\partial t} = \frac{V(t_i,S_j)-V(t_{i-1},S_j)}{\delta t}$$
$$\Gamma =\frac{\partial^2 V}{\partial S^2}= \frac{V(t_i,S_{j+1})-2V(t_i,S_j)+V(t_i,S_{j-1})}{(\delta S)^2}$$
$$ \Delta =\frac{\partial V}{\partial S}= \frac{V(t_i,S_{j+1})-V(t_i,S_{j-1})}{2\delta S}$$
Thus, we have:
$$V(t_{i-1},S_j)=(1-r \delta t) V(t_i,S_j)+\frac{1}{2}\sigma^2 \delta t S_j^2 \frac{V(t_i,S_{j+1})+2V(t_i,S_j)+V(t_i,S_{j-1})}{(\delta S)^2} + (r-q) \delta t S_j \frac{V(t_i,S_{j+1})-V(t_i,S_{j-1})}{2\delta S}$$
$$V(t_{i-1},S_j)= a_j V(t_i,S_{j-1}) + b_j V(t_i, S_j) + c_j V(t_i, S_{j+1})$$
with $a_j=\frac{1}{2} \delta t (\sigma^2 j^2-(r-q)j) $, $b_j=1-\delta t (\sigma^2 j^2 +r)$ and $c_j=\frac{1}{2} \delta t (\sigma^2 j^2+(r-q)j)$ using $S_j=j \delta S$.

We set boundaries conditions:
$$ \forall i, V(t_i,0) = 0 $$
$$ \forall i, V(t_i, S_j) \underset{S_j \rightarrow \infty}{\sim} S_j - K e^{-r(T-t_i)}$$
$$ \forall j, V(T, S_j) = \max(S_j - K, 0)$$

In [ ]:
/**
 * @brief Prices American or European call/put options using the finite difference method.
 *
 * @param call_put A string indicating the type of option: "call" or "put".
 * @param am_eu A string indicating the exercise style: "American" or "European".
 * @param sigma The volatility of the underlying asset (annualized standard deviation of returns).
 * @param K The strike price of the option.
 * @param T The time to maturity of the option (in years).
 * @param St The current price of the underlying asset.
 * @param r The risk-free interest rate (annualized, continuously compounded).
 * @param q The dividend yield of the underlying asset (annualized, continuously compounded).
 *
 * @return double The price of the American or European call/put option.
 *
 * @throws runtime_error If `call_put` is not "call" or "put", or if `am_eu` is not "American" or "European".
 *
 * @details
 * This function uses an explicit finite difference method to solve the Black-Scholes PDE for option pricing.
 * The grid is defined by `n_t` time steps and `n_S` price steps, with a maximum asset price of \( S_{\text{max}} = St(1 + 2\sigma\sqrt{T}) \).
 * The stability condition for the explicit method is checked:
 * \[
 * \Delta t \cdot \sigma^2 \cdot n_S^2 \leq 1
 * \]
 * If this condition is violated, a warning is printed.
 *
 * For European options, the PDE is solved using backward induction without early exercise.
 * For American options, the solution includes an early exercise check at each step.
 *
 * The coefficients for the finite difference scheme are:
 * \[
 * a = 0.5 \cdot \Delta t \cdot (\sigma^2 j^2 - (r-q)j)
 * \]
 * \[
 * b = 1 - \Delta t \cdot (\sigma^2 j^2 + r)
 * \]
 * \[
 * c = 0.5 \cdot \Delta t \cdot (\sigma^2 j^2 + (r-q)j)
 * \]
 * The option value at each grid point is updated as:
 * \[
 * V_{i-1,j} = a V_{i,j-1} + b V_{i,j} + c V_{i,j+1}
 * \]
 * For American options, the value is also compared to the immediate exercise value.
 */
double PDE(string call_put, string am_eu, double sigma, double K, double T, double St, double r, double q) {
    // Finite difference method for American and European options
    double S_max = St * (1 + 2 * sigma * sqrt(T));
    int n_t = 1000; // Number of time points
    int n_S = 50;   // Number of price points
    double dt = T / (n_t - 1);
    double dS = S_max / (n_S - 1);
    double V[n_t][n_S];

    // Stability check
    if (dt * sigma * sigma * n_S * n_S > 1) {
        cout << "Change the parameters in order to assure the stability." << endl;
    }

    // Initial and boundary conditions
    if (call_put == "call") {
        for (int i = 0; i < n_t; i++) {
            V[i][0] = 0.;
            V[i][n_S - 1] = S_max - K * exp(-r * (T - i * dt));
        }
        for (int j = 0; j < n_S; j++) {
            V[n_t - 1][j] = max(j * dS - K, 0.);
        }
    } else if (call_put == "put") {
        for (int i = 0; i < n_t; i++) {
            V[i][0] = K * exp(-r * (T - i * dt));
            V[i][n_S - 1] = 0.;
        }
        for (int j = 0; j < n_S; j++) {
            V[n_t - 1][j] = max(K - j * dS, 0.);
        }
    } else {
        throw runtime_error("Please choose between call and put");
    }

    // Backward computation
    double a, b, c;
    if (am_eu == "European") {
        for (int i = n_t - 1; i > 0; i--) {
            for (int j = 1; j < n_S - 1; j++) {
                a = 0.5 * dt * (sigma * sigma * j * j - (r - q) * j);
                b = 1 - dt * (sigma * sigma * j * j + r);
                c = 0.5 * dt * (sigma * sigma * j * j + (r - q) * j);
                V[i - 1][j] = a * V[i][j - 1] + b * V[i][j] + c * V[i][j + 1];
            }
        }
    } else if (am_eu == "American") {
        if (call_put == "call") {
            for (int i = n_t - 1; i > 0; i--) {
                for (int j = 1; j < n_S - 1; j++) {
                    a = 0.5 * dt * (sigma * sigma * j * j - (r - q) * j);
                    b = 1 - dt * (sigma * sigma * j * j + r);
                    c = 0.5 * dt * (sigma * sigma * j * j + (r - q) * j);
                    V[i - 1][j] = max(a * V[i][j - 1] + b * V[i][j] + c * V[i][j + 1], j * dS - K);
                }
            }
        } else if (call_put == "put") {
            for (int i = n_t - 1; i > 0; i--) {
                for (int j = 1; j < n_S - 1; j++) {
                    a = 0.5 * dt * (sigma * sigma * j * j - (r - q) * j);
                    b = 1 - dt * (sigma * sigma * j * j + r);
                    c = 0.5 * dt * (sigma * sigma * j * j + (r - q) * j);
                    V[i - 1][j] = max(a * V[i][j - 1] + b * V[i][j] + c * V[i][j + 1], K - j * dS);
                }
            }
        }
    } else {
        throw runtime_error("Please choose between American and European");
    }
    return V[0][int(round(St / dS))];
}

In [11]:
cout << "PDE European call price: "+to_string(PDE("call", "European", sigma, K, T, St, r, q)) << endl;
cout << "PDE European put price: "+to_string(PDE("put", "European", sigma, K, T, St, r, q)) << endl;

cout << "PDE American call price: "+to_string(PDE("call", "American", sigma, K, T, St, r, q)) << endl;
cout << "PDE American put price: "+to_string(PDE("put", "American", sigma, K, T, St, r, q)) << endl;

PDE European call price: 0.061975
PDE European put price: 34.533300
PDE American call price: 1.461421
PDE American put price: 34.533300
